### **Miniproyecto1**

Santiago Montoya Londoño

Hanna Isabella Colorado M.

**cc:** 1037672299

**cc:** 1000408557

In [ ]:
#Descarga archivo base de dadtos

#https://drive.google.com/file/d/1cNMil1P60z-XrqExnnKvmggmXWhI3glZ/view?usp=drive_link -> s14.mat
#https://drive.google.com/file/d/1Fu3HL-cB64lTfuZS8iPhIfaJIsl1OszD/view?usp=drive_link -> s15.mat
#https://drive.google.com/file/d/1cBN0CYdrsUsVO77PyNq-Jkyh5pc0FI22/view?usp=drive_link -> s16.mat
#https://drive.google.com/file/d/1bMObQcIFrNalRmD1LaNuqVSTeqaeGqR3/view?usp=drive_link -> s17.mat
!gdown '1cNMil1P60z-XrqExnnKvmggmXWhI3glZ'
!gdown '1Fu3HL-cB64lTfuZS8iPhIfaJIsl1OszD'
!gdown '1cBN0CYdrsUsVO77PyNq-Jkyh5pc0FI22'
!gdown '1bMObQcIFrNalRmD1LaNuqVSTeqaeGqR3'

Downloading...
From: https://drive.google.com/uc?id=1cNMil1P60z-XrqExnnKvmggmXWhI3glZ
To: /content/s14.mat
100% 80.6M/80.6M [00:00<00:00, 103MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Fu3HL-cB64lTfuZS8iPhIfaJIsl1OszD
To: /content/s15.mat
100% 80.7M/80.7M [00:00<00:00, 179MB/s]
Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1cBN0CYdrsUsVO77PyNq-Jkyh5pc0FI22 

Downloading...
From: https://drive.google.com/uc?id=1bMObQcIFrNalRmD1LaNuqVSTeqaeGqR3
To: /content/s17.mat
100% 80.7M/80.7M [00:00<00:00, 229MB/s]


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sc
import scipy.signal as signal
import seaborn as sns
import pandas as pd

In [ ]:
def spectralpower(biosignal, filtertype, band, fs):
  wn=2*band/fs
  filter=signal.butter(7, wn, btype=filtertype, analog=False, fs=fs, output='sos')
  filtered = signal.sosfilt(filter, biosignal)

  f, t, Zxx = signal.stft(filtered, fs, nperseg=512) # generar STFT
  power=(np.abs(Zxx)**2)/len(Zxx)
  power=np.sort(power, axis=None)
  power=power[-10:]
  return power

def caracteristicas(signal1):   #40x6528
  GSR=signal1[36,:]
  xGSR=[]
  media=np.mean(GSR)
  xGSR.append(media)
  derivative=np.mean(np.diff(GSR))
  xGSR.append(derivative)

  power=spectralpower(GSR,'lowpass',2.4, 128)
  xGSR=np.concatenate((xGSR,power))

  xPleth=[]
  pleth=signal1[38,:]
  media=np.mean(pleth)
  des=np.std(pleth)
  xPleth.append(media)
  xPleth.append(des)
  media=np.mean(np.diff(pleth))
  des=np.std(np.diff(pleth))
  xPleth.append(media)
  xPleth.append(des)

  power=spectralpower(pleth,'bandpass',np.array([0.2, 0.3]), 128)
  xPleth=np.concatenate((xPleth,power))

  xRes=[]
  res=signal1[37,:]
  derivative=np.mean(np.diff(res))
  xRes.append(derivative)

  peaks,_=signal.find_peaks(res,distance=128, height=0)
  time=[]
  for i in range(len(peaks)-1):
    time.append((peaks[i+1]-peaks[i])/128)
  media=np.mean(time)
  xRes.append(media)

  power=spectralpower(res,'lowpass',2.4, 128)
  xRes=np.concatenate((xRes,power))

  xTemp=[]
  temp=signal1[39,:]
  media=np.mean(temp)
  derivative=np.mean(np.diff(temp))
  xTemp.append(media)
  xTemp.append(derivative)

  power=spectralpower(temp,'bandpass',np.array([0.1, 0.2]), 128)
  xTemp=np.concatenate((xTemp,power))

  xEOG=[]
  xEMG=[]
  hEOG=signal1[32,:]
  vEOG=signal1[33,:]

  zEMG=signal1[34,:]
  tEMG=signal1[35,:]

  M=[hEOG,vEOG,zEMG,tEMG]
  for i in range(4):
    energy=np.sum(np.abs(M[i])**2)
    media=np.mean(M[i])
    variance=np.std(M[i])**2
    if i==0 or i==1:
      xEOG.append(energy)
      xEOG.append(media)
      xEOG.append(variance)
    else:
      xEMG.append(energy)
      xEMG.append(media)
      xEMG.append(variance)

  Fp1=signal1[0,:]
  Fp2=signal1[16,:]

  xEEG=[]
  bandas=np.array([[8,12],[12,30],[30,100]])
  for i in range(3):
    aux1=spectralpower(Fp1,'bandpass',bandas[i],128)
    aux2=spectralpower(Fp2,'bandpass',bandas[i],128)
    asimetry=np.abs(aux1-aux2)
    xEEG=np.concatenate((xEEG,asimetry))
  xTrial=np.hstack((xGSR,xPleth,xRes,xTemp,xEOG,xEMG,xEEG))
  return xTrial

In [ ]:
datafull=pd.DataFrame([])
for i in range(14,18):
  datos=sc.loadmat(f's{i}.mat')
  signals=datos['data']
  etiquetas=datos['labels']
  valencia=etiquetas[:,0]

  emocion=[]
  for n in valencia:
    if n<=4:
      emocion.append('Negativo')
    if n>4 and n<7:
      emocion.append('Neutro')
    if n>=7:
      emocion.append('Positivo')

  P=np.zeros((40,92))
  for i in range(40):
    P[i,:]=caracteristicas(signals[i,:,:])
  dataframe=pd.DataFrame(P)
  dataframe['Emocion']=emocion

  datafull=pd.concat([datafull,dataframe])
datafull=datafull.reset_index()
datafull

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,index,0,1,2,3,4,5,6,7,8,...,83,84,85,86,87,88,89,90,91,Emocion
0,0,-18005.683435,-8.677097,1.013148e+06,1.025629e+06,1.074571e+06,1.367917e+06,2.211068e+06,3.049811e+06,3.065904e+06,...,7.182084e-06,9.148706e-06,1.172425e-05,1.340746e-05,2.259001e-05,2.228289e-05,2.381127e-05,2.436900e-05,4.269278e-05,Positivo
1,1,-8794.650746,-3.715479,3.873666e+05,3.935669e+05,4.134574e+05,5.582798e+05,9.353378e+05,1.086986e+06,1.301715e+06,...,1.878743e-04,2.341424e-04,2.677827e-04,3.228892e-04,5.542446e-04,8.432189e-04,8.719355e-04,9.320010e-04,1.301624e-03,Neutro
2,2,-1331.408630,0.360847,2.354429e+04,2.731418e+04,3.494548e+04,4.731772e+04,5.503736e+04,6.860747e+04,7.491371e+04,...,1.472889e-06,1.884362e-06,2.134097e-06,2.284675e-06,5.614110e-06,6.717441e-06,8.599692e-06,9.874653e-06,1.451447e-05,Positivo
3,3,3491.283824,0.470205,4.509419e+04,5.043383e+04,6.488768e+04,6.550028e+04,8.838457e+04,1.149172e+05,1.435423e+05,...,2.196778e-04,2.350688e-04,2.796237e-04,3.299953e-04,6.386819e-04,8.984570e-04,9.855013e-04,1.025293e-03,1.483919e-03,Positivo
4,4,-3736.386024,-1.386627,1.922613e+04,2.178912e+04,3.056869e+04,3.769286e+04,4.470393e+04,5.568385e+04,5.949361e+04,...,1.216148e-05,1.419085e-05,1.542479e-05,1.880412e-05,2.215572e-05,2.869670e-05,3.161717e-05,4.776016e-05,5.547685e-05,Negativo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,35,5204.994820,1.298265,5.993729e+04,7.862485e+04,9.672037e+04,1.123824e+05,1.173350e+05,1.243760e+05,1.324127e+05,...,7.403217e-07,1.111291e-06,7.451672e-07,8.728313e-07,1.489492e-06,4.358906e-06,4.907805e-06,3.503238e-06,3.559600e-06,Negativo
156,36,2975.633960,1.408379,6.524916e+02,7.083481e+02,1.121859e+03,3.153543e+03,4.156603e+03,6.617796e+03,1.215281e+04,...,8.719878e-06,9.453669e-06,1.209131e-05,1.392848e-05,2.551160e-05,3.088522e-05,3.603420e-05,3.637079e-05,5.822170e-05,Negativo
157,37,7866.527529,2.349866,7.239846e+04,7.651857e+04,9.369866e+04,1.063916e+05,1.214118e+05,1.565672e+05,1.994897e+05,...,2.872424e-07,3.244073e-07,3.788534e-07,4.157885e-07,7.192217e-07,1.859614e-06,2.015518e-06,1.933047e-06,2.060198e-06,Negativo
158,38,-2334.049943,-0.487896,1.337780e+04,1.594861e+04,1.888148e+04,2.186510e+04,2.415110e+04,2.534149e+04,2.827295e+04,...,4.275135e-06,4.838911e-06,5.501343e-06,6.677718e-06,1.175620e-05,1.755561e-05,1.939974e-05,1.991299e-05,2.863159e-05,Positivo


In [ ]:
section=datafull.iloc[:,[1,2,3,4,93]]
sns.pairplot(section,hue="Emocion")